# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import nltk
import re
import string
import os
import wordninja
import json
import torch
from itertools import groupby
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from data_preprocessing import *
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# 데이터 로드

In [2]:
data=[]
with open('C:/Users/user/Desktop/bilm-tf-master/textdataset/News_Category_Dataset_v3.json', 'r') as f:
    for line in f:
        content = json.loads(line)
        data.append(content)
df = pd.DataFrame(data=data)

# 전처리

In [3]:
df = df[~df['short_description'].apply(lambda x: len(x)==0)]

#중복되는 카테고리 통합
df["category"] = df.category.replace(
    {
        "THE WORLDPOST": "WORLD NEWS",
        "WORLDPOST": "WORLD NEWS",
        "ARTS": "ARTS & CULTURE",
        "CULTURE & ARTS": "ARTS & CULTURE",
        "HEALTHY LIVING": "WELLNESS",
        "QUEER VOICES": "VOICES",
        "LATINO VOICES": "VOICES",
        "BLACK VOICES": "VOICES",
        "STYLE": "STYLE & BEATUY",
        "GREEN": "ENVIRONMENT",
        "TASTE": "FOOD & DRINK",
        "MONEY": "BUSINESS",
        "PARENTING": "PARENTS"
    }
)

df = df[['headline', 'short_description','category']]
df['input_data']= df.apply(lambda x: str(x['headline']) + str(x['short_description']), axis=1)

df['clean_text']=df['input_data'].apply(clean_text)

class0=list(df[df['category'] == 'POLITICS'].sample(1000, random_state=1)['clean_text'])
class1=list(df[df['category'] == 'WELLNESS'].sample(1000, random_state=1)['clean_text'])
class2=list(df[df['category'] == 'ENTERTAINMENT'].sample(1000, random_state=1)['clean_text'])
class3=list(df[df['category'] == 'PARENTS'].sample(1000, random_state=1)['clean_text'])
class4=list(df[df['category'] == 'VOICES'].sample(1000, random_state=1)['clean_text'])
class5=list(df[df['category'] == 'STYLE & BEAUTY'].sample(1000, random_state=1)['clean_text'])
class6=list(df[df['category'] == 'TRAVEL'].sample(1000, random_state=1)['clean_text'])
class7=list(df[df['category'] == 'FOOD & DRINK'].sample(1000, random_state=1)['clean_text'])
class8=list(df[df['category'] == 'WORLD NEWS'].sample(1000, random_state=1)['clean_text'])
class9=list(df[df['category'] == 'BUSINESS'].sample(1000, random_state=1)['clean_text'])

X_list=class0+class1+class2+class3+class4+class5+class6+class7+class8+class9

y=[]
for i in range(10):
    y+=[i]*1000
    
clr_x_data=[]
pattern = '[^a-z ]'
for sen in X_list:
    clr_x_data.append(re.sub(pattern, ' ', sen))
    
X_list=[]
for sen in clr_x_data:
    X_list.append(' '.join(sen.split()))
    
train_idx=[]
for i in range(10):
    train_idx+=[j+i*1000 for j in range(800)]

test_idx=[]
for i in range(10):
    test_idx+=[j+800+i*1000 for j in range(200)]
    
x_train=[]
y_train=[]
for i in train_idx:
    x_train.append(X_list[i])
    y_train.append(y[i])

x_test=[]
y_test=[]
for i in test_idx:
    x_test.append(X_list[i])
    y_test.append(y[i])
    
to_txt=x_train+x_test
y=list(y_train)+list(y_test)

In [4]:
to_txt

['gop congressman trump should apologize to obama over wiretapping claimsthe comments from oklahomas tom cole are the strongest republican admonition of trumps baseless allegations to date',
 'the gop tax plan is not as simple as they saya business tax cut could open a big loophole for wealthy filers to use',
 'obama says us should be proud of economic progressa new report released on friday showed an eightyearlow in unemployment',
 'trump administration sued for delaying protections for endangered beean executive order has kept protections for the rusty patched bumble bee species from taking effect',
 'trump agrees with obama tweet but twitter users suspect hes trollingthe president said he totally agreed with a obama tweet asking for comprehensive immigration reform',
 'friday talking points double standards and hypocrisywere going to begin today with a rather loaded question how much attention do you think the media should be paying towards a presidential nominee who is right now ge

# 사전 학습 BERT 가져오기

In [5]:
def split_continuous_integers(lst):                                      #연속된 정수리스트를 split 해주는 함수
    for k, g in groupby(enumerate(lst), lambda i_x:i_x[0]-i_x[1]):
        yield list(map(itemgetter(1), g))
        
def add_previous_number(lst):                                            #최초 서브토큰 인덱스 추가
    return [[sub_lst[0] - 1] + sub_lst for sub_lst in lst]

def bert_word_embedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화
    indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Python list를 PyTorch tensor로 변환하기 
    tokens_tensor = torch.tensor([indexed_text])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors )
    token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
    token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
    token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])
    
    token_vecs_sum = []
    for token in token_embeddings:                #인코더의 마지막 4개의 은닉 상태를 합쳐 최종 벡터(4개 합친 것이 성능이 가장 좋음)
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(np.array(sum_vec))
    
    token_vecs_sum=np.array(token_vecs_sum)
    #서브토큰을 결합해 단어에 대한 임베딩 벡터를 얻자
    subword_indices = [i for i, token in enumerate(tokenized_text) if '##' in token]
    index_list = add_previous_number(list(split_continuous_integers(subword_indices))) #index_list는 서브토큰에 해당하는 인덱스를 한 리스트에 묶어줌
    
    new_token_vecs_sum = []
    last_index = 0

    for subword_inx_list in index_list:
        # 이전 인덱스와 현재 인덱스 그룹 사이의 벡터를 추가합니다.
        new_token_vecs_sum.extend(token_vecs_sum[last_index:subword_inx_list[0]])

        # 현재 인덱스 그룹에 해당하는 벡터의 평균을 계산하고 추가합니다.
        avg_vecs = np.mean(token_vecs_sum[subword_inx_list], axis=0)
        new_token_vecs_sum.append(avg_vecs)

        # 마지막 인덱스를 업데이트합니다.
        last_index = subword_inx_list[-1] + 1

    # 마지막 인덱스 이후의 벡터를 추가합니다.
    new_token_vecs_sum.extend(token_vecs_sum[last_index:])

    # 결과를 numpy array로 변환합니다.
    new_token_vecs_sum = np.array(new_token_vecs_sum[1:-1])
    
    return new_token_vecs_sum

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 미리 학습된 모델(가중치) 불러오기
model = BertModel.from_pretrained('bert-base-uncased')

text = to_txt[5]

marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화

indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [1] * len(tokenized_text)

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_text])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors )
    
token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])

token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(np.array(sum_vec))
    

token_vecs_sum=np.array(token_vecs_sum)

token_vecs_sum=bert_word_embedding(text)
token_vecs_sum.shape

(47, 768)

임베딩

In [7]:
x_train=to_txt[:8000]
x_test=to_txt[8000:]

In [ ]:
except_lst=[]
for i in range(len(x_train)):
    try:
        text = to_txt[i]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(26, 768)
(24, 768)
(19, 768)
(24, 768)
(25, 768)
(47, 768)
(19, 768)
(27, 768)
(32, 768)
(13, 768)
(28, 768)
(20, 768)
(33, 768)
(23, 768)
(24, 768)
(29, 768)
(29, 768)
(34, 768)
(17, 768)
(12, 768)
(21, 768)
(21, 768)
(28, 768)
(30, 768)
(22, 768)
(28, 768)
(30, 768)
(20, 768)
(26, 768)
(30, 768)
(36, 768)
(19, 768)
(26, 768)
(15, 768)
(10, 768)
(16, 768)
(28, 768)
(48, 768)
(16, 768)
(19, 768)
(25, 768)
(22, 768)
(20, 768)
(26, 768)
(17, 768)
(32, 768)
(12, 768)
(29, 768)
(31, 768)
(40, 768)
(34, 768)
(33, 768)
(15, 768)
(30, 768)
(28, 768)
(31, 768)
(26, 768)
(31, 768)
(23, 768)
(23, 768)
(24, 768)
(14, 768)
(22, 768)
(22, 768)
(24, 768)
(20, 768)
(15, 768)
(30, 768)
(20, 768)
(22, 768)
(35, 768)
(20, 768)
(29, 768)
(24, 768)
(29, 768)
(27, 768)
(20, 768)
(25, 768)
(29, 768)
(18, 768)
(30, 768)
(27, 768)
(27, 768)
(27, 768)
(28, 768)
(39, 768)
(29, 768)
(16, 768)
(32, 768)
(24, 768)
(27, 768)
(27, 768)
(26, 768)
(28, 768)
(25, 768)
(25, 768)
(29, 768)
(31, 768)
(26, 768)
(19, 768)


(55, 768)
(16, 768)
(17, 768)
(19, 768)
(10, 768)
(30, 768)
(40, 768)
(24, 768)
(29, 768)
(17, 768)
(23, 768)
(29, 768)
(11, 768)
(14, 768)
(36, 768)
(33, 768)
(17, 768)
(16, 768)
(15, 768)
(26, 768)
(25, 768)
(32, 768)
(30, 768)
(13, 768)
(29, 768)
(9, 768)
(10, 768)
(35, 768)
(57, 768)
(26, 768)
(18, 768)
(33, 768)
(31, 768)
(37, 768)
(47, 768)
(17, 768)
(20, 768)
(26, 768)
(13, 768)
(16, 768)
(16, 768)
(25, 768)
(18, 768)
(18, 768)
(16, 768)
(26, 768)
(32, 768)
(36, 768)
(43, 768)
(26, 768)
(24, 768)
(25, 768)
(51, 768)
(15, 768)
(13, 768)
(26, 768)
(31, 768)
(17, 768)
(15, 768)
(70, 768)
(19, 768)
(19, 768)
(18, 768)
(11, 768)
(28, 768)
(56, 768)
(23, 768)
(20, 768)
(22, 768)
(26, 768)
(28, 768)
(38, 768)
(12, 768)
(35, 768)
(15, 768)
(17, 768)
(30, 768)
(58, 768)
(29, 768)
(27, 768)
(20, 768)
(23, 768)
(49, 768)
(11, 768)
(15, 768)
(27, 768)
(17, 768)
(28, 768)
(44, 768)
(16, 768)
(13, 768)
(16, 768)
(24, 768)
(22, 768)
(55, 768)
(32, 768)
(30, 768)
(29, 768)
(23, 768)
(25, 768)
(

(36, 768)
(45, 768)
(52, 768)
(35, 768)
(58, 768)
(66, 768)
(11, 768)
(55, 768)
(49, 768)
(38, 768)
(24, 768)
(29, 768)
(26, 768)
(69, 768)
(37, 768)
(35, 768)
(39, 768)
(24, 768)
(27, 768)
(25, 768)
(54, 768)
(34, 768)
(34, 768)
(61, 768)
(32, 768)
(28, 768)
(35, 768)
(39, 768)
(39, 768)
(25, 768)
(29, 768)
(53, 768)
(26, 768)
(27, 768)
(35, 768)
(43, 768)
(26, 768)
(35, 768)
(49, 768)
(41, 768)
(29, 768)
(48, 768)
(30, 768)
(15, 768)
(22, 768)
(33, 768)
(27, 768)
(64, 768)
(33, 768)
(33, 768)
(41, 768)
(48, 768)
(30, 768)
(28, 768)
(37, 768)
(63, 768)
(54, 768)
(52, 768)
(28, 768)
(34, 768)
(33, 768)
(29, 768)
(62, 768)
(34, 768)
(29, 768)
(18, 768)
(38, 768)
(49, 768)
(48, 768)
(29, 768)
(37, 768)
(31, 768)
(19, 768)
(34, 768)
(26, 768)
(27, 768)
(43, 768)
(14, 768)
(44, 768)
(47, 768)
(35, 768)
(26, 768)
(25, 768)
(28, 768)
(71, 768)
(46, 768)
(39, 768)
(10, 768)
(43, 768)
(16, 768)
(31, 768)
(37, 768)
(27, 768)
(21, 768)
(36, 768)
(30, 768)
(49, 768)
(36, 768)
(41, 768)
(38, 768)


(33, 768)
(30, 768)
(29, 768)
(32, 768)
(27, 768)
(12, 768)
(22, 768)
(32, 768)
(18, 768)
(28, 768)
(55, 768)
(46, 768)
(16, 768)
(21, 768)
(16, 768)
(17, 768)
(37, 768)
(42, 768)
(24, 768)
(44, 768)
(23, 768)
(21, 768)
(30, 768)
(42, 768)
(30, 768)
(20, 768)
(36, 768)
(19, 768)
(24, 768)
(52, 768)
(24, 768)
(22, 768)
(19, 768)
(33, 768)
(25, 768)
(35, 768)
(12, 768)
(52, 768)
(46, 768)
(24, 768)
(54, 768)
(21, 768)
(40, 768)
(29, 768)
(45, 768)
(26, 768)
(20, 768)
(25, 768)
(32, 768)
(21, 768)
(29, 768)
(15, 768)
(17, 768)
(20, 768)
(62, 768)
(31, 768)
(21, 768)
(16, 768)
(33, 768)
(19, 768)
(57, 768)
(29, 768)
(19, 768)
(30, 768)
(31, 768)
(20, 768)
(25, 768)
(28, 768)
(23, 768)
(24, 768)
(34, 768)
(33, 768)
(26, 768)
(38, 768)
(32, 768)
(32, 768)
(42, 768)
(26, 768)
(33, 768)
(23, 768)
(26, 768)
(20, 768)
(27, 768)
(26, 768)
(19, 768)
(56, 768)
(29, 768)
(19, 768)
(20, 768)
(24, 768)
(60, 768)
(60, 768)
(30, 768)
(24, 768)
(48, 768)
(28, 768)
(34, 768)
(27, 768)
(20, 768)
(43, 768)


(29, 768)
(27, 768)
(23, 768)
(34, 768)
(26, 768)
(26, 768)
(31, 768)
(30, 768)
(13, 768)
(33, 768)
(30, 768)
(30, 768)
(30, 768)
(38, 768)
(27, 768)
(23, 768)
(25, 768)
(29, 768)
(33, 768)
(29, 768)
(28, 768)
(32, 768)
(32, 768)
(21, 768)
(25, 768)
(25, 768)
(31, 768)
(27, 768)
(28, 768)
(27, 768)
(30, 768)
(43, 768)
(29, 768)
(35, 768)
(15, 768)
(27, 768)
(27, 768)
(33, 768)
(47, 768)
(35, 768)
(38, 768)
(35, 768)
(35, 768)
(45, 768)
(28, 768)
(33, 768)
(30, 768)
(40, 768)
(25, 768)
(32, 768)
(31, 768)
(30, 768)
(31, 768)
(31, 768)
(32, 768)
(31, 768)
(29, 768)
(25, 768)
(31, 768)
(13, 768)
(35, 768)
(17, 768)
(28, 768)
(32, 768)
(29, 768)
(30, 768)
(28, 768)
(35, 768)
(33, 768)
(16, 768)
(28, 768)
(29, 768)
(33, 768)
(27, 768)
(64, 768)
(26, 768)
(31, 768)
(34, 768)
(27, 768)
(42, 768)
(27, 768)
(29, 768)
(29, 768)
(31, 768)
(32, 768)
(32, 768)
(29, 768)
(33, 768)
(53, 768)
(34, 768)
(42, 768)
(27, 768)
(32, 768)
(27, 768)
(33, 768)
(30, 768)
(25, 768)
(25, 768)
(34, 768)
(32, 768)


(32, 768)
(19, 768)
(62, 768)
(38, 768)
(35, 768)
(34, 768)
(51, 768)
(30, 768)
(27, 768)
(27, 768)
(31, 768)
(60, 768)
(30, 768)
(35, 768)
(38, 768)
(31, 768)
(31, 768)
(21, 768)
(19, 768)
(18, 768)
(30, 768)
(25, 768)
(13, 768)
(20, 768)
(45, 768)
(64, 768)
(31, 768)
(35, 768)
(27, 768)
(28, 768)
(31, 768)
(45, 768)
(54, 768)
(28, 768)
(51, 768)
(54, 768)
(10, 768)
(29, 768)
(30, 768)
(39, 768)
(46, 768)
(44, 768)
(36, 768)
(42, 768)
(29, 768)
(43, 768)
(21, 768)
(38, 768)
(39, 768)
(20, 768)
(55, 768)
(45, 768)
(69, 768)
(71, 768)
(36, 768)
(31, 768)
(11, 768)
(30, 768)
(41, 768)
(25, 768)
(53, 768)
(20, 768)
(27, 768)
(30, 768)
(15, 768)
(23, 768)
(35, 768)
(26, 768)
(32, 768)
(24, 768)
(32, 768)
(29, 768)
(28, 768)
(30, 768)
(30, 768)
(23, 768)
(28, 768)
(29, 768)
(46, 768)
(15, 768)
(33, 768)
(35, 768)
(36, 768)
(50, 768)
(43, 768)
(34, 768)
(48, 768)
(64, 768)
(32, 768)
(25, 768)
(31, 768)
(27, 768)
(43, 768)
(28, 768)
(28, 768)
(33, 768)
(26, 768)
(14, 768)
(33, 768)
(51, 768)


(53, 768)
(21, 768)
(25, 768)
(13, 768)
(22, 768)
(19, 768)
(30, 768)
(29, 768)
(17, 768)
(18, 768)
(33, 768)
(28, 768)
(39, 768)
(27, 768)
(31, 768)
(42, 768)
(9, 768)
(32, 768)
(28, 768)
(20, 768)
(16, 768)
(16, 768)
(13, 768)
(55, 768)
(12, 768)
(18, 768)
(30, 768)
(15, 768)
(16, 768)
(45, 768)
(25, 768)
(19, 768)
(10, 768)
(29, 768)
(17, 768)
(19, 768)
(22, 768)
(20, 768)
(19, 768)
(23, 768)
(32, 768)
(19, 768)
(23, 768)
(15, 768)
(13, 768)
(25, 768)
(26, 768)
(19, 768)
(52, 768)
(22, 768)
(15, 768)
(19, 768)
(54, 768)
(32, 768)
(25, 768)
(26, 768)
(19, 768)
(21, 768)
(44, 768)
(23, 768)
(45, 768)
(19, 768)
(23, 768)
(25, 768)
(18, 768)
(50, 768)
(53, 768)
(27, 768)
(12, 768)
(41, 768)
(48, 768)
(28, 768)
(24, 768)
(13, 768)
(27, 768)
(24, 768)
(25, 768)
(18, 768)
(10, 768)
(15, 768)
(14, 768)
(34, 768)
(55, 768)
(10, 768)
(32, 768)
(21, 768)
(18, 768)
(33, 768)
(25, 768)
(25, 768)
(23, 768)
(25, 768)
(19, 768)
(7, 768)
(13, 768)
(20, 768)
(38, 768)
(17, 768)
(8, 768)
(17, 768)
(24

(24, 768)
(17, 768)
(22, 768)
(17, 768)
(22, 768)
(28, 768)
(16, 768)
(27, 768)
(15, 768)
(11, 768)
(31, 768)
(33, 768)
(14, 768)
(17, 768)
(42, 768)
(15, 768)
(35, 768)
(20, 768)
(21, 768)
(28, 768)
(20, 768)
(20, 768)
(45, 768)
(41, 768)
(16, 768)
(28, 768)
(46, 768)
(32, 768)
(57, 768)
(26, 768)
(23, 768)
(27, 768)
(26, 768)
(19, 768)
(21, 768)
(25, 768)
(19, 768)
(19, 768)
(13, 768)
(63, 768)
(16, 768)
(19, 768)
(27, 768)
(28, 768)
(32, 768)
(47, 768)
(31, 768)
(24, 768)
(20, 768)
(40, 768)
(27, 768)
(23, 768)
(17, 768)
(22, 768)
(26, 768)
(18, 768)
(33, 768)
(33, 768)
(33, 768)
(57, 768)
(23, 768)
(32, 768)
(55, 768)
(26, 768)
(32, 768)
(33, 768)
(26, 768)
(23, 768)
(19, 768)
(28, 768)
(27, 768)
(27, 768)
(21, 768)
(28, 768)
(28, 768)
(21, 768)
(29, 768)
(17, 768)
(22, 768)
(51, 768)
(41, 768)
(19, 768)
(29, 768)
(16, 768)
(30, 768)
(30, 768)
(22, 768)
(23, 768)
(26, 768)
(19, 768)
(22, 768)
(21, 768)
(39, 768)
(32, 768)
(19, 768)
(15, 768)
(22, 768)
(16, 768)
(16, 768)
(28, 768)


(29, 768)
(41, 768)
(74, 768)
(19, 768)
(19, 768)
(24, 768)
(31, 768)
(29, 768)
(31, 768)
(10, 768)
(28, 768)
(25, 768)
(41, 768)
(35, 768)
(29, 768)
(22, 768)
(47, 768)
(34, 768)
(39, 768)
(22, 768)
(35, 768)
(41, 768)
(15, 768)
(27, 768)
(49, 768)
(27, 768)
(23, 768)
(30, 768)
(16, 768)
(28, 768)
(35, 768)
(52, 768)
(45, 768)
(28, 768)
(18, 768)
(31, 768)
(59, 768)
(43, 768)
(29, 768)
(32, 768)
(30, 768)
(31, 768)
(32, 768)
(30, 768)
(50, 768)
(45, 768)
(32, 768)
(31, 768)
(15, 768)
(33, 768)
(34, 768)
(52, 768)
(48, 768)
(44, 768)
(13, 768)
(10, 768)
(25, 768)
(20, 768)
(21, 768)
(24, 768)
(53, 768)
(50, 768)
(60, 768)
(31, 768)
(29, 768)
(22, 768)
(15, 768)
(33, 768)
(37, 768)
(51, 768)
(46, 768)
(18, 768)
(29, 768)
(27, 768)
(26, 768)
(37, 768)
(26, 768)
(46, 768)
(29, 768)
(27, 768)
(25, 768)
(19, 768)
(38, 768)
(24, 768)
(29, 768)
(17, 768)
(30, 768)
(117, 768)
(33, 768)
(20, 768)
(32, 768)
(38, 768)
(18, 768)
(27, 768)
(21, 768)
(20, 768)
(42, 768)
(39, 768)
(40, 768)
(30, 768)

In [11]:
import pickle

print('제외된 문서의 수: ',len(except_lst))

with open('except_lst.pkl', 'wb') as file:
    pickle.dump(except_lst, file)

print("except_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_lst를 피클 파일로 저장했습니다.


In [12]:
except_test_lst=[]

for i in range(len(x_test)):
    try:
        text = to_txt[i+8000]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_test_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(23, 768)
(87, 768)
(44, 768)
(31, 768)
(18, 768)
(27, 768)
(12, 768)
(20, 768)
(33, 768)
(29, 768)
(28, 768)
(31, 768)
(23, 768)
(24, 768)
(31, 768)
(27, 768)
(25, 768)
(22, 768)
(25, 768)
(59, 768)
(35, 768)
(17, 768)
(14, 768)
(14, 768)
(20, 768)
(10, 768)
(24, 768)
(30, 768)
(28, 768)
(49, 768)
(41, 768)
(27, 768)
(16, 768)
(38, 768)
(32, 768)
(26, 768)
(16, 768)
(27, 768)
(27, 768)
(25, 768)
(16, 768)
(28, 768)
(34, 768)
(35, 768)
(20, 768)
(19, 768)
(29, 768)
(25, 768)
(43, 768)
(23, 768)
(24, 768)
(21, 768)
(40, 768)
(28, 768)
(39, 768)
(26, 768)
(21, 768)
(25, 768)
(28, 768)
(19, 768)
(28, 768)
(71, 768)
(13, 768)
(31, 768)
(42, 768)
(25, 768)
(12, 768)
(22, 768)
(20, 768)
(26, 768)
(24, 768)
(24, 768)
(31, 768)
(24, 768)
(23, 768)
(21, 768)
(23, 768)
(27, 768)
(28, 768)
(31, 768)
(18, 768)
(19, 768)
(24, 768)
(19, 768)
(19, 768)
(23, 768)
(19, 768)
(32, 768)
(30, 768)
(22, 768)
(34, 768)
(25, 768)
(24, 768)
(22, 768)
(33, 768)
(27, 768)
(22, 768)
(23, 768)
(34, 768)
(23, 768)


(16, 768)
(39, 768)
(22, 768)
(30, 768)
(25, 768)
(25, 768)
(19, 768)
(22, 768)
(30, 768)
(24, 768)
(24, 768)
(26, 768)
(31, 768)
(25, 768)
(65, 768)
(22, 768)
(45, 768)
(24, 768)
(23, 768)
(32, 768)
(48, 768)
(21, 768)
(27, 768)
(29, 768)
(20, 768)
(22, 768)
(33, 768)
(59, 768)
(19, 768)
(28, 768)
(41, 768)
(28, 768)
(43, 768)
(12, 768)
(19, 768)
(33, 768)
(35, 768)
(23, 768)
(24, 768)
(19, 768)
(22, 768)
(22, 768)
(29, 768)
(30, 768)
(18, 768)
(24, 768)
(26, 768)
(31, 768)
(38, 768)
(44, 768)
(20, 768)
(20, 768)
(30, 768)
(17, 768)
(24, 768)
(12, 768)
(32, 768)
(18, 768)
(35, 768)
(14, 768)
(31, 768)
(38, 768)
(23, 768)
(26, 768)
(53, 768)
(26, 768)
(10, 768)
(36, 768)
(48, 768)
(41, 768)
(39, 768)
(28, 768)
(22, 768)
(22, 768)
(14, 768)
(31, 768)
(21, 768)
(23, 768)
(21, 768)
(36, 768)
(14, 768)
(28, 768)
(56, 768)
(23, 768)
(58, 768)
(12, 768)
(18, 768)
(26, 768)
(15, 768)
(31, 768)
(40, 768)
(30, 768)
(24, 768)
(32, 768)
(28, 768)
(31, 768)
(32, 768)
(20, 768)
(31, 768)
(28, 768)


(24, 768)
(28, 768)
(39, 768)
(31, 768)
(20, 768)
(19, 768)
(16, 768)
(24, 768)
(26, 768)
(31, 768)
(28, 768)
(23, 768)
(28, 768)
(15, 768)
(28, 768)
(24, 768)
(19, 768)
(22, 768)
(50, 768)
(16, 768)
(17, 768)
(17, 768)
(18, 768)
(14, 768)
(28, 768)
(34, 768)
(27, 768)
(31, 768)
(12, 768)
(40, 768)
(29, 768)
(23, 768)
(39, 768)
(14, 768)
(16, 768)
(31, 768)
(25, 768)
(21, 768)
(34, 768)
(21, 768)
(16, 768)
(45, 768)
(30, 768)
(31, 768)
(26, 768)
(53, 768)
(25, 768)
(26, 768)
(22, 768)
(27, 768)
(22, 768)
(48, 768)
(32, 768)
(30, 768)
(21, 768)
(21, 768)
(20, 768)
(29, 768)
(26, 768)
(19, 768)
(18, 768)
(19, 768)
(27, 768)
(44, 768)
(24, 768)
(14, 768)
(25, 768)
(25, 768)
(26, 768)
(18, 768)
(31, 768)
(22, 768)
(21, 768)
(28, 768)
(19, 768)
(74, 768)
(21, 768)
(33, 768)
(13, 768)
(26, 768)
(23, 768)
(57, 768)
(22, 768)
(24, 768)
(13, 768)
(30, 768)
(27, 768)
(31, 768)
(26, 768)
(23, 768)
(18, 768)
(19, 768)
(31, 768)
(24, 768)
(31, 768)
(21, 768)
(17, 768)
(17, 768)
(21, 768)
(15, 768)


In [13]:
import pickle

print('제외된 문서의 수: ',len(except_test_lst))

with open('except_test_lst.pkl', 'wb') as file:
    pickle.dump(except_test_lst, file)

print("except_test_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_test_lst를 피클 파일로 저장했습니다.
